In [1]:
import cv2
import numpy as np

# turn on camera
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Unable to open camera")
    exit()

# To store coordinates
recorded_coords = []

while True:
    ret, frame = cap.read()
    if not ret:
        print("Unable to read frame")
        break

    # RGB to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # blue usually stays around H 100-140，S 150-255，V 0-255
    lower_blue = np.array([100, 150, 0])
    upper_blue = np.array([140, 255, 255])

    # generate cover
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # find edge
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected_coord = None
    if contours:
        # select the largest blue area
        largest_contour = max(contours, key=cv2.contourArea)
        if cv2.contourArea(largest_contour) > 500:  # area threshold
            x, y, w, h = cv2.boundingRect(largest_contour)
            center_x = x + w // 2
            center_y = y + h // 2
            detected_coord = (center_x, center_y)

            # draw area and middle point
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1)
            cv2.putText(frame, f"({center_x}, {center_y})", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # show
    cv2.imshow("Blue Object Detection", frame)
    cv2.imshow("Mask", mask)

    # q to quit, enter to record
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == 13:  # Enter 键
        if detected_coord is not None:
            recorded_coords.append(detected_coord)
            print("Captured coordinate:", detected_coord)
        else:
            print("no blue item")

        break

cap.release()
cv2.destroyAllWindows()

print("All recorded coordinates:", recorded_coords)

Captured coordinate: (341, 305)
All recorded coordinates: [(341, 305)]


In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Vision-based Pick & Place Demo
--------------------------------
Use（recorded_coords），
Convert camera coordinates to robots（set z=179）。
procedure：
  1. read recorded_coords（use first coordinates）；
  2. use matrix H to convert (x, y)；
  3. set grab point：[x, y, 179, -179.46, -6.69, 95.57]；
  4. run grab process：go to zero position, open graper, go to destination, close graper, lift, go to placement position, place, zero, close graper.
"""

import time
import numpy as np
import json
from pymycobot import MyCobot320Socket

def getIpConfig():
    # Open and read the JSON file
    with open('env/ipconfig.json', 'r') as file:
        data = json.load(file)

    # read the ip and port info
    ip_address = data['ip']
    netport = data['port']

    return ip_address, netport

def convert_camera_to_robot(camera_coord, H):

    u, v = camera_coord
    point_h = np.array([u, v, 1.0])
    robot_h = H.dot(point_h)
    robot_h /= robot_h[2]
    return (robot_h[0], robot_h[1])


try:
    captured_camera_coord = recorded_coords[0]
except (NameError, IndexError):
    print("Error: recorded_coords undefined")
    exit(1)

print("Captured camera coordinate from vision:", captured_camera_coord)

H = np.array([
    [6.60782927e-04,  2.48469514e+00, -5.96091742e+02],
    [3.82506417e-01,  4.06164160e-01, -2.18163280e+02],
    [9.21284300e-05, -5.55189057e-03,  1.00000000e+00]
])

robot_xy = convert_camera_to_robot(captured_camera_coord, H)
print("Converted robot (x, y):", robot_xy)

pick_z = 165
pick_orientation = [-179.46, -6.69, 95.57]
pick_coords = [robot_xy[0], robot_xy[1], pick_z] + pick_orientation
print("Pick coordinates:", pick_coords)

place_coords = [-329.1, 104.6, 179.1, -179.46, -6.69, 95.57]

ip_address, netport = getIpConfig()
mc = MyCobot320Socket(ip_address, netport)
time.sleep(1)

mc.focus_all_servos()
time.sleep(1)

print("\n---> set_gripper_mode(0) => pass-through")
ret_mode = mc.set_gripper_mode(0)
print("     Return code:", ret_mode)
time.sleep(1)

home_angles = [0, 0, 0, 0, 0, 0]
print("\n---> Move to home position:", home_angles)
mc.send_angles(home_angles, 30)
time.sleep(3)

speed = 30  

print("\n---> Open gripper")
mc.set_gripper_state(0, 100)
time.sleep(2)

print("\n---> Move to pick coordinates")
mc.send_coords(pick_coords, speed, 1)
time.sleep(3)

print("\n---> Close gripper to grasp block")
mc.set_gripper_state(1, 100)
time.sleep(2)

pick_coords_ascend = [pick_coords[0], pick_coords[1], pick_coords[2] + 50] + pick_orientation
print("\n---> Ascend after grasping (z + 50)")
mc.send_coords(pick_coords_ascend, speed, 1)
time.sleep(3)

print("\n---> Move to place coordinates")
mc.send_coords(place_coords, speed, 1)
time.sleep(3)

print("\n---> Open gripper to release block")
mc.set_gripper_state(0, 100)
time.sleep(2)

print("\n---> Return to home position")
mc.send_angles(home_angles, 30)
time.sleep(3)

print("\n---> Close gripper (final state)")
mc.set_gripper_state(1, 100)
time.sleep(2)

print("\nPick & Place sequence completed.\n")


Captured camera coordinate from vision: (341, 305)
Converted robot (x, y): (np.float64(-244.69399128726792), np.float64(-54.616838705601985))
Pick coordinates: [np.float64(-244.69399128726792), np.float64(-54.616838705601985), 165, -179.46, -6.69, 95.57]

---> set_gripper_mode(0) => pass-through
     Return code: -1

---> Move to home position: [0, 0, 0, 0, 0, 0]

---> Open gripper

---> Move to pick coordinates

---> Close gripper to grasp block

---> Ascend after grasping (z + 50)

---> Move to place coordinates

---> Open gripper to release block

---> Return to home position

---> Close gripper (final state)

Pick & Place sequence completed.

